# Flowsheet initialization
## ROK model = M5; NGL Feed: EF-Basin

In [1]:
# simulation case specifications
model_code = 5
case_name = 'Bakken'

In [2]:
from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

# Import idaes model serializer to store initialized model
from idaes.core.util import model_serializer as ms

from pyomo.environ import (Constraint,
                           ConstraintList,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Set,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value,
                           minimize)

from src.unit_initialization import create_flowsheet, \
                                    define_models, \
                                    define_arcs, \
                                    set_unit_model_variables, \
                                    initialize_flowsheet, \
                                    set_scaling_factors,\
                                    update_model_after_initialization,\
                                    vapor_only_to_vapor_liquid_reformulate,\
                                    H106_inlet_vapor_reformulate, \
                                    replace_heater_heat_duty_constraint_with_bounds, \
                                    update_model_for_optimization, \
                                    unfix_DOFs_pre_optimization, \
                                    fix_DOFs_post_optimization

In [3]:
m = create_flowsheet(model_code)

In [4]:
M_catalyst = 1167.003367 # kg

## Define equipment and connections

In [5]:
# define unit models
define_models(m, catalyst_mass = M_catalyst)
#define connections
define_arcs(m)

## Define inlet compositions

In [6]:
import pandas as pd

inlet_df = pd.read_csv('NGL_compositions.csv')

inlet_composition_dict = {}

for col in inlet_df.columns:
    if col == case_name:
        for i,r in inlet_df.iterrows():
            if r[col] == 0.0:
                inlet_composition_dict[r['Species']] = 1e-6
            else:
                inlet_composition_dict[r['Species']] = round(r[col],4)

inlet_flow_rate = 481.3888889

dehydro_conv_dict = {'ethane':0.3566,
                     'propane':0.6632,
                     'nbutane':0.5188}

## Define constraints and set-points for equipment

In [7]:
set_unit_model_variables(m, model_code=model_code, feed_flow_rate = inlet_flow_rate, 
                         feed_temp = 308.0, feed_pressure = 700000.0,
                         inlet_composition_dict = inlet_composition_dict,
                         dehydro_conv_dict = dehydro_conv_dict)

## Scale model components

In [8]:
if model_code == 2 or model_code == 3:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-2, inlet_composition_dict = inlet_composition_dict)
elif model_code == 4 or model_code == 5:
    set_scaling_factors(m,flow_mol_scaling_factor = 1e-3, inlet_composition_dict = inlet_composition_dict)
else:
    pass

## Sequence for initialization

In [9]:
from pyomo.network import SequentialDecomposition
seq = SequentialDecomposition()
seq.options.select_tear_method = "heuristic"
seq.options.tear_method = "Wegstein"
seq.options.iterLim = 5

# Using the SD tool
G = seq.create_graph(m)
heuristic_tear_set = seq.tear_set_arcs(G, method="heuristic")
order = seq.calculation_order(G)

for o in heuristic_tear_set:
    print(o.name)
print("Order of initialization")
for o in order:
    print(o[0].name)    
    
def function(unit):
    unit.initialize(outlvl=idaeslog.INFO_HIGH)  

fs.s01
Order of initialization
fs.H101
fs.R101
fs.H102
fs.S101
fs.H103
fs.R102
fs.H104
fs.T102
fs.H105
fs.F101
fs.H106
fs.F102
fs.T104
fs.C101
fs.T105
fs.M102
fs.S102
fs.C102
fs.M101


## Read-in initialization data from initialization with M5 and Bakken feed

In [10]:
init_file_name = "./initialization_files/CISTAR_unit_initialization_{}_M{}.json.gz".format(case_name, model_code)
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.07416248321533203,
 'etime_read_dict': 0.17183732986450195,
 'etime_read_suffixes': 0.006053447723388672}

## Add post-initialization constraints

In [11]:
update_model_after_initialization(m)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)
vapor_only_to_vapor_liquid_reformulate(m.fs.T102)

Liquid phase flow is zero. Translator block fs.T102 liquid phase composition equations are being modified...

Degenerate constraints removed and liquid phase composition values set to 1e-8

Translator block fs.T102 liquid phase composition equations already modified, no degenerate constraints remaining.



## Read-in converged flowsheet solution using M5 and Bakken feed

In [12]:
init_file_name = "./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3))
ms.from_json(m, fname=init_file_name)

{'etime_load_file': 0.07401680946350098,
 'etime_read_dict': 0.1745152473449707,
 'etime_read_suffixes': 0.0}

In [13]:
DOF_initial = degrees_of_freedom(m)
print("The final DOF of initialized flowsheet is {0}".format(DOF_initial))

## Dataframes to record multiple zone output

In [14]:
import pandas as pd

In [15]:
cols = ['region','butene', 'pentene', 'hexene', 'heptene', 'octene', 'nonene']
outlet_conc_df = pd.DataFrame(columns=cols)

LHV_cols = ['region','LHV_kJ_mol','MW_fuel','LHV_kJ_s']
LHV_df = pd.DataFrame(columns=LHV_cols)

In [16]:
conc_list = []
for col in cols:
    if col == 'region':
        conc_list.append(case_name)
    else:
        conc_list.append(m.fs.F102.liq_outlet.flow_mol_phase_comp[0.0, 'Liq', col]())

outlet_conc_df.loc[len(outlet_conc_df.index)] = conc_list

print(outlet_conc_df)

   region    butene    pentene     hexene    heptene     octene     nonene
0  Bakken  6.593976  12.499718  20.333122  24.197909  29.876602  25.110165


## Flowsheet initialization for multiple feed compositions

In [17]:
for col in inlet_df.columns:
    if col == 'Species':
        pass
    elif col == 'EF-Basin':
        region = col
        case_name_current = col
        for i,r in inlet_df.iterrows():
            m.fs.M101.feed.mole_frac_comp[0, r['Species']].unfix()
            if r[col] == 0.0:
                m.fs.M101.feed.mole_frac_comp[0, r['Species']].fix(1e-6)
            else:
                m.fs.M101.feed.mole_frac_comp[0, r['Species']].fix(round(r[col],4))

        m.fs.C102.inlet.flow_mol.unfix()
        m.fs.M101.recycle.flow_mol.unfix()
        for k,v in inlet_composition_dict.items():
            m.fs.C102.inlet.mole_frac_comp[0, k].unfix()
            m.fs.M101.recycle.mole_frac_comp[0, k].unfix()
        
        m.fs.H106.outlet.temperature.unfix()
        m.fs.H106.outlet.pressure.unfix()

        print('\n Region = {} \n'.format(col))
        solver = SolverFactory('ipopt')
        solver.options = {'tol': 1e-6,
                          'bound_push': 1e-8,
                          'max_iter': 500
                         }
        solve_status = solver.solve(m, tee=True)

        m.fs.H106.outlet.temperature.fix()
        m.fs.H106.outlet.pressure.fix()
        
        ms.to_json(m, fname="./initialization_files/CISTAR_solve_constrained_{}_M{}_purge_{}.json.gz".format(case_name_current, model_code,round(m.fs.S102.split_fraction[0, "purge"](),3)))
        
        conc_list = []
        for col in cols:
            if col == 'region':
                conc_list.append(region)
            else:
                conc_list.append(m.fs.F102.liq_outlet.flow_mol_phase_comp[0.0, 'Liq', col]())

        outlet_conc_df.loc[len(outlet_conc_df.index)] = conc_list
        
        print("====================  Region = {}  ======================".format(region))
        print(outlet_conc_df)
    else:
        pass


 Region = EF-Basin 

Ipopt 3.13.2: tol=1e-06
bound_push=1e-08
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran cod